## 데이터 전처리

In [117]:
%pip install langchainhub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
12401.30s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
  Obtaining dependency information for langchainhub from https://files.pythonhosted.org/packages/9c/99/4306b825b9462c824fd6d8dcb959309fb5e654c0d7b6473c5379388a82e9/langchainhub-0.1.14-py3-none-any.whl.metadata
  Obtaining dependency information for types-requests<3.0.0.0,>=2.31.0.2 from https://files.pythonhosted.org/packages/ce/ca/82c7f75616c524856488cece6b37e459de626cad49b2a24a0b571c20be06/types_requests-2.31.0.20240218-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/a2/73/a68704750a7679d0b6d

In [86]:
import os
import pandas as pd

# 디렉토리 경로
directory = "/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/sick"

# 모든 CSV 파일을 읽어들여 데이터를 통합합니다.
all_data = pd.DataFrame()
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        filepath = os.path.join(directory, filename)
        data = pd.read_csv(filepath)
        all_data = pd.concat([all_data, data])

# 증상과 병명 열만 남깁니다.
all_data = all_data[['증상', '병명']]

# 누락된 값이 있는 행을 제거합니다.
all_data = all_data.dropna(subset=['증상', '병명'])

# 결과를 새로운 CSV 파일로 저장합니다.
output_filepath = "/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/combined_data.csv"
all_data.to_csv(output_filepath, index=False)

print("통합된 데이터를 저장했습니다:", output_filepath)


통합된 데이터를 저장했습니다: /Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/combined_data.csv


## 데이터에서 형태소 분석을 통해 증상을 명사로만 뽑아서 시도해봄.



In [179]:
%pip install konlpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
17870.30s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.


In [183]:
import pandas as pd
from konlpy.tag import Okt

# CSV 파일 경로
csv_file_path = "/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/combined_data.csv"

# CSV 파일 읽기
data = pd.read_csv(csv_file_path)

# Okt 형태소 분석기 초기화
okt = Okt()

# 형태소 분석 및 명사 추출 함수 정의
def extract_nouns(text):
    nouns = okt.nouns(text)  # 형태소 분석기를 사용하여 명사만 추출
    return " ".join(nouns)  # 추출된 명사들을 하나의 문자열로 결합하여 반환

# 모든 증상 데이터에 대해 형태소 분석 및 명사 추출 적용
data["증상"] = data["증상"].apply(extract_nouns)

# 결과 출력
print(data["증상"])


0       다른 사람 권리 무시 행동 양식 반복 지속 이 반복 범법 행위 참여 거나 연루 다른...
1       통증 압통 및 종창 부종 전형 증상 급 성기 체중 서기 정도 손상 경우 순간 인대 ...
2       자궁 경관 용종 대부분 증상 정기 검진 시 의사 발견 증상 경우 생리 중간 부정 기...
3       복통 보통 오른쪽 왼쪽 중 한쪽 경우 대부분 몇 시간 안 통증 복부 불편 감 지속 ...
4       대부분 무증상 경우 그 외 비정상 질 출혈 성교 시 통증 및 출혈 질 분비물 증가 ...
                              ...                        
1240      손발 저리 증상 의학 감각 이상 주파수 진동 감각 과 느낌 주관 간혹 통증 동반 기도
1241    무 조짐 편두통 편두통 보통 전구 증상 조짐 두통 및 동반 증상 해소 기 후유증 상...
1242    만성 췌장염 환자 통증 지속 간 적일 수 통증 수도 점 급성 췌장염 차이 통증 전형...
1243    망막색소변성증 대표 가지 증상 야맹증 시야협착 야맹증 야간 시력 저하 증상 망막색소...
1244                  증상 신체 과열 것 땀 동반 과민증 보통 증상 시작 장기간 지속
Name: 증상, Length: 1245, dtype: object


In [184]:
data

,증상,병명
0,다른 사람 권리 무시 행동 양식 반복 지속 이 반복 범법 행위 참여 거나 연루 다른...,반사회적인격장애
1,통증 압통 및 종창 부종 전형 증상 급 성기 체중 서기 정도 손상 경우 순간 인대 ...,발목염좌
2,자궁 경관 용종 대부분 증상 정기 검진 시 의사 발견 증상 경우 생리 중간 부정 기...,자궁내경관용종
3,복통 보통 오른쪽 왼쪽 중 한쪽 경우 대부분 몇 시간 안 통증 복부 불편 감 지속 ...,배란통
4,대부분 무증상 경우 그 외 비정상 질 출혈 성교 시 통증 및 출혈 질 분비물 증가 ...,자궁경부비정형세포
...,...,...
1240,손발 저리 증상 의학 감각 이상 주파수 진동 감각 과 느낌 주관 간혹 통증 동반 기도,"손발저림,감각없음"
1241,무 조짐 편두통 편두통 보통 전구 증상 조짐 두통 및 동반 증상 해소 기 후유증 상...,편두통
1242,만성 췌장염 환자 통증 지속 간 적일 수 통증 수도 점 급성 췌장염 차이 통증 전형...,만성췌장염
1243,망막색소변성증 대표 가지 증상 야맹증 시야협착 야맹증 야간 시력 저하 증상 망막색소...,망막색소변성증


In [186]:
# 저장할 CSV 파일 경로
output_csv_path = "/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/nouns.csv"

# 데이터를 CSV 파일로 저장
data.to_csv(output_csv_path, index=False)


# retriever tuning 
### embedding  = jhgan/ko-sbert-nli

### .env file 에 자신의 lang smith api key를 정의.
#### .env file엔 Lang smith를 연결하는 [API_KEY]와 gemini api인  [GOOGLE_API_KEY] 두개를 정의.

# Lang smith 에 연결 (모니터링을 위함)

In [87]:
from IPython.display import Markdown

In [88]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [89]:
import os

# 디버깅을 위한 프로젝트명을 기입합니다.
os.environ["retriever_tuning"] = "RAG TUTORIAL"

# tracing 을 위해서는 아래 코드의 주석을 해제하고 실행합니다.
# os.environ["LANGCHAIN_TRACING_V2"] = true

In [90]:
pip install -U langchain

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
11415.97s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.


In [91]:
# 환경 변수 설정
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "retriever_tuning"

# API 키는 .env 파일에 저장되어 있다고 가정하고 불러옴
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"] = os.getenv("API_KEY")

# 구글 gemini api 

In [92]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 GOOGLE_API_KEY 가져오기
google_api_key = os.getenv("GOOGLE_API_KEY")

# 필요한 경우, 가져온 API 키를 사용하여 환경 변수 설정
os.environ["GOOGLE_API_KEY"] = google_api_key


In [93]:
%pip install -U --quiet langchain-google-genai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
11424.02s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.


In [94]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Gemini test

In [95]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("열이 나고 목이 아프고 기침을 합니다")
Markdown(result.content)

**증상:**

* 열
* 목 아픔
* 기침

**가능한 원인:**

* 독감
* 감기
* 편도선염
* 후두염
* 기관지염
* 폐렴

**치료:**

증상의 근본 원인에 따라 치료가 달라집니다.

**일반적인 치료법:**

* **휴식:** 충분한 휴식을 취하세요.
* **수분 공급:** 많은 수분을 섭취하세요.
* **목 아픔 완화:** 따뜻한 소금물로 가글을 하거나 로진지를 빨아보세요.
* **기침 완화:** 기침 억제제 또는 점액제거제를 복용하세요.
* **열 낮춤:** 아세트아미노펜 또는 이부프로펜과 같은 해열제를 복용하세요.

**의사의 진료가 필요한 경우:**

* 열이 38.9°C(102°F) 이상인 경우
* 목이 심하게 아프거나 삼키기 어려운 경우
* 기침이 심하거나 호흡곤란이 있는 경우
* 증상이 3일 이상 지속되는 경우
* 증상이 악화되거나 새로운 증상이 나타나는 경우

**예방:**

* 감기와 독감 예방접종을 받으세요.
* 자주 손을 씻으세요.
* 아픈 사람과 가까이 접촉하는 것을 피하세요.
* 흡연을 피하세요.
* 건강한 식습관과 수면습관을 유지하세요.

**주의 사항:**

진단과 치료는 의료 전문가가 해야 합니다. 자가 치료를 하지 마세요.

# vectoer store : chromadb

In [96]:
%pip install -U --quiet langchain tiktoken pypdf sentence_transformers chromadb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
11439.79s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.


In [97]:
pip install chromadb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
11448.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.


In [98]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader

## retriever tuning : Ensemble Retriever

In [99]:
%pip install -q langchain pypdf sentence-transformers langchain-openai --upgrade --quiet  rank_bm25 > /dev/null

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
11455.77s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.


In [100]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.embeddings import HuggingFaceEmbeddings

In [101]:
# loader = CSVLoader("/Users/jangminsoo/Desktop/dev/Medical-project/crawling/qna_data/소아청소년_QNA.csv", encoding='utf-8')
# docs = loader.load()


# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=50)

# texts = text_splitter.split_documents(docs)


# from langchain.embeddings import HuggingFaceEmbeddings

# model_name = "jhgan/ko-sroberta-multitask"
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': True}
# hf = HuggingFaceEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs
# )

# docsearch = Chroma.from_documents(texts, hf)

In [197]:
import pandas as pd

# CSV 파일 경로
csv_file_path = "/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/combined_data.csv"

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

In [198]:
df

,증상,병명
0,"다른 사람의 권리를 무시하는 무책임한 행동 양식을 반복적, 지속적으로 보인다. 많은...",반사회적인격장애
1,"통증, 압통 및 종창과 부종이 전형적인 증상으로 급성기에는 체중을 싣고 서기가 힘들...",발목염좌
2,자궁내경관 용종은 대부분 증상이 없고 정기검진 시 의사에 의해 우연히 발견된다. 증...,자궁내경관용종
3,"복통은 보통 심하지 않고, 오른쪽 또는 왼쪽 중 한쪽만 아픈 경우가 많다. 대부분은...",배란통
4,"대부분은 무증상인 경우가 많으며, 그 외 비정상적인 질 출혈이나 성교 시 통증 및 ...",자궁경부비정형세포
...,...,...
1240,"흔히 ""손발이 저리다""라는 증상은 의학적으로 감각이상(paresthesia)라고 하...","손발저림,감각없음"
1241,1) 무조짐 편두통(migraine without aura)편두통은 보통 전구증상-...,편두통
1242,"만성 췌장염 환자의 통증은 지속적 간헐적일 수 있으며, 통증이 없을 수도 있다는 점...",만성췌장염
1243,망막색소변성증의 대표적인 2가지 증상으로 야맹증과 시야협착이 있다. 야맹증은 야간 ...,망막색소변성증


In [104]:
df[df['병명']=='감기']['증상']

986    감기 바이러스에 노출된 지 1~3일 후에 증상이 나타난다. 증상은 감기 바이러스가 ...
Name: 증상, dtype: object

In [204]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import CSVLoader

# CSV 파일을 읽어와서 문서로 변환합니다.
loader = CSVLoader("/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/combined_data.csv")
docs = loader.load_and_split()

# 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(docs)




In [205]:
docs[1]

Document(page_content='증상: 통증, 압통 및 종창과 부종이 전형적인 증상으로 급성기에는 체중을 싣고 서기가 힘들 정도이다. 심한 손상의 경우 다치는 순간 인대가 끊어지는 파열음을 들을 수 있는 경우도 있다. 그러나 증상의 심한 정도가 반드시 손상 정도와 일치하는 것은 아니다. 심한 손상으로 발목관절 주변 인대가 파열이 되거나 관절의 탈구가 동반되는 경우 관절의 불안정성이 동반될 수 있으나 다친 직후에는 통증으로 인한 근육 경직으로 확인이 어려울 수 있다. 초기에는 통증, 압통, 종창이 주증상이며 부기가 가라앉으면서 멍이 발목아래쪽으로 생기는 경과를 거친다. 골절 또는 다양한 관절 내 손상이 동반될 수 있어 이에 따른 증상을 나타내기도 한다.\n병명: 발목염좌', metadata={'source': '/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/combined_data.csv', 'row': 1})

### embedding 진행
#### embedding model : jhgan/ko-sbert-nli

In [207]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


In [208]:
docs[1]

Document(page_content='증상: 통증, 압통 및 종창과 부종이 전형적인 증상으로 급성기에는 체중을 싣고 서기가 힘들 정도이다. 심한 손상의 경우 다치는 순간 인대가 끊어지는 파열음을 들을 수 있는 경우도 있다. 그러나 증상의 심한 정도가 반드시 손상 정도와 일치하는 것은 아니다. 심한 손상으로 발목관절 주변 인대가 파열이 되거나 관절의 탈구가 동반되는 경우 관절의 불안정성이 동반될 수 있으나 다친 직후에는 통증으로 인한 근육 경직으로 확인이 어려울 수 있다. 초기에는 통증, 압통, 종창이 주증상이며 부기가 가라앉으면서 멍이 발목아래쪽으로 생기는 경과를 거친다. 골절 또는 다양한 관절 내 손상이 동반될 수 있어 이에 따른 증상을 나타내기도 한다.\n병명: 발목염좌', metadata={'source': '/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/combined_data.csv', 'row': 1})

In [209]:
# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 5

chroma_vectorstore = Chroma.from_documents(texts, hf)
chroma_retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 5})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.7, 0.3]
)

In [211]:
docs = ensemble_retriever.invoke("오늘 저희 어머니가 집에 혼자계시다 입을 비롯하여 안면근육이 약간돌아가는걸 느껴 병원에 갔다오셨다는데 병원에서 뇌경색이 왔다가 그냥 지나갔다고 했다는군요. 침을 좀 맞는걸 권유했다는데맞는말인가요? 저희어머니가 걱정안시키려고 거짓말하시는거같아서요. 정말 그럴수도있는건가요?")
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content)
    print("-"*100)

{'row': 549, 'source': '/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/combined_data.csv'}
:
증상: 부모, 특히 어머니가 옆에 있어야 안심하고 헤어져 있을 때는 어머니나 자신에게 나쁜 일이 생겨서 서로 보지 못하게 될까 불안해한다. 그래서 자꾸 전화를 걸어서 어머니의 존재를 확인하게 된다. 혼자 집에 있지 못하고, 잠을 잘 때 어머니가 옆에 있어야 안심하며, 꿈은 어머니와 헤어지거나, 어머니나 자신에게 사고가 나는 등의 분리를 주제로 한 꿈을 많이 꾼다. 그 외에도 복통, 두통 등의 잦은 신체 증상을 보여 무의식적으로 어머니의 관심과 사랑을 구하기도 한다.
병명: 분리불안장애
----------------------------------------------------------------------------------------------------
{'source': '/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/combined_data.csv', 'row': 435}
:
증상: 지속되는 심한 인후통이 나타나고, 같은 쪽의 귀에서 이통(귀 통증)이 나타난다. 환자는 급성 증상을 보이고, 지속되는 발열과 오한, 위약감, 오심 등을 동반한다. 연하 곤란(삼키기 어려움)으로 심하면 침을 삼키기 힘들어져 입 밖으로 흘리게 되고, 숨을 내뱉을 때 역겨운 냄새가 난다.익돌근(pterygoid muscle)에 염증이 생기면 개구장애(턱의 근육이 경련을 일으켜 입이 벌어지지 않는 상태)가 발생한다. 말하는 것이 힘들어지며, 비음이나 통증을 줄이기 위해 입을 다물지 못하고 말하게 되는데, 이 때 마치 뜨거운 감자를 입에 물고 있는 듯한 목소리(hot potato voice)가 나타난다. 탈수

In [194]:
# 각각의 BM25 Retriever와 Chroma Retriever에서 결과를 가져옵니다.
bm25_docs = bm25_retriever.invoke("엄지 발가락 통증이 있습니다")
chroma_docs = chroma_retriever.invoke("엄지 발가락 통증이 있습니다")

# 가져온 문서들을 출력합니다.
print("BM25 Retriever 결과:")
for doc in bm25_docs:
    print(doc.metadata)
    print(":")
    print(doc.page_content)
    print("-"*100)

print("Chroma Retriever 결과:")
for doc in chroma_docs:
    print(doc.metadata)
    print(":")
    print(doc.page_content)
    print("-"*100)

# Ensemble Retriever를 사용하여 최종 결과를 가져옵니다.
ensemble_docs = ensemble_retriever.invoke("엄지 발가락 통증이 있습니다")

# 최종 결과를 출력합니다.
print("종합 결과:")
for doc in ensemble_docs:
    print(doc.metadata)
    print(":")
    print(doc.page_content)
    print("-"*100)



BM25 Retriever 결과:
{'source': '/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/nouns.csv', 'row': 567}
:
증상: 가장 증상 제 중족 발가락 관절 안쪽 돌출 부위 건막류 의 통증 이 부위 신발 자극 염증 통증 발생 이차 두 세 발가락 발바닥 쪽 살이 생기 통증 발생 경우 두 발가락 엄지 발가락 겹 살 및 압박 피부 궤양 발생 관절 탈구 기도 새끼 발가락 쪽 관절 돌출 변형 생기 기도
병명: 무지외반증
----------------------------------------------------------------------------------------------------
{'source': '/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/nouns.csv', 'row': 260}
:
증상: 개 발가락 중 엄지발가락 가장 발생 오른발 엄지 때 가장 압박 부위 이기 때문 처음 엄지발가락 외측 내 약간 통증 이내 마찰 더 진물 육아 조직 염증 혈관 섬유 조직 증식 덩어리 이 증식 발톱 주위 곪기 시작 진행 냄새 통증 정상 보행
병명: 내향성발톱
----------------------------------------------------------------------------------------------------
{'source': '/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/nouns.csv', 'row': 798}
:
증상: 골격계 환자 호소 증상 환자 외관 통해 의심 수 골격계 이상 가장 특징 마르고 키 매우 몸 팔다리 비정상 것 환자 

In [195]:
# 각각의 BM25 Retriever와 Chroma Retriever에서 결과를 가져옵니다.
bm25_docs = bm25_retriever.invoke("콧물이 있고 기침과 열이 있습니다")
chroma_docs = chroma_retriever.invoke("콧물이 있고 기침과 열이 있습니다")

# 각 엔진에서 상위 n개의 문서를 가져옵니다.
top_bm25_docs = bm25_docs[:5]  # n은 원하는 상위 문서의 개수입니다.
top_chroma_docs = chroma_docs[:5]  # n은 원하는 상위 문서의 개수입니다.

# 선택된 상위 문서를 결합하여 최종 결과를 생성합니다.
ensemble_docs = top_bm25_docs + top_chroma_docs

# 최종 결과를 출력합니다.
print("종합 결과:")
for doc in ensemble_docs:
    print(doc.metadata)
    print(":")
    print(doc.page_content)
    print("-"*100)



종합 결과:
{'source': '/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/nouns.csv', 'row': 1244}
:
증상: 증상 신체 과열 것 땀 동반 과민증 보통 증상 시작 장기간 지속
병명: 열과민증
----------------------------------------------------------------------------------------------------
{'source': '/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/nouns.csv', 'row': 458}
:
증상: 신경학 증상 발생 수 의식 저하 비정상 불수 운동 등 여러 증상 수 뇌사 상태 수도
병명: 허혈성뇌손상
----------------------------------------------------------------------------------------------------
{'source': '/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/nouns.csv', 'row': 403}
:
위협 정도 증상 초래 수 주의 진단 치료 신경계 증상 편두통 포함 두통 불쾌감 피로 감 초조 우울증 등 매우 입술 주위 두드러기
----------------------------------------------------------------------------------------------------
{'source': '/Users/jangminsoo/Desktop/dev/Medical-project/LLM_embedding_change/ko-sroberta-multitask/jhgan/

### prompt 적용

In [160]:
# RAG prompt
from langchain import hub
prompt = hub.pull("rlm/rag-prompt-llama")

In [161]:
from langchain.chains import RetrievalQA

In [166]:
# RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=ensemble_retriever,
    return_source_documents = True,
    chain_type_kwargs={"prompt": prompt}
)

In [111]:
# gemini = ChatGoogleGenerativeAI(model="gemini-pro", temperature = 0) # 답변의 일관성을 위한 0 값

# chain = RunnableMap({
#     "context": lambda x: ensemble_retriever.invoke(x['question']),
#     "question": lambda x: x['question']
# }) | prompt | gemini

In [163]:
question = "열과 기침이 있는데 무슨 증상인가요?"
qa_chain.invoke({"query": question})

{'query': '열과 기침이 있는데 무슨 증상인가요?',
 'result': '열과 기침이 있는 증상에 대해서는 주어진 텍스트에서 정보를 찾을 수 없습니다.'}

In [176]:
question = "귀에 통증 있고 고름이 있습니다 무슨 증상인가요?"
result = qa_chain({"query": question})
result["result"]

'귀에 통증이 있고 고름이 잡히는 증상은 종기의 증상입니다. 종기는 초기에는 단단하고 만지면 아픈 붉은 결절로 시작하여 점차 커지면서 통증이 심해지고 고름이 잡힙니다. 후기에는 손가락으로 가볍게 눌러도 움푹 패일 정도로 물렁물렁해집니다.'

In [177]:
for i in result['source_documents']:
    print(i.page_content)

증상: 귀의 통증이 나타난다.
병명: 이통
증상: 일반적으로 귓바퀴가 머리로부터 대략 1.7~2cm 떨어져 있는데, 돌출귀는 이보다 더 멀리 떨어져 있다. 심한 경우에는 이륜이 전혀 구부러져 있지 않아 귓바퀴가 조가비(조개의 껍데기) 모양일 때도 있다. 양쪽 귀에 동시에 나타나는 경우가 많다.
병명: 돌출귀
증상: 초기에는 단단하고 만지면 아픈 붉은 결절로 시작하여 점차 커지면서 통증이 심해지고 고름이 잡힌다(화농). 후기에는 손가락으로 가볍게 눌러도 움푹 패일 정도로 물렁물렁해지고, 완전히 곪으면 고름이 터져 배출된다. 2~3주 후에 흉터나 색소 침착을 남기면서 치유가 되며, 색소 침착은 수개월에 걸쳐 옅어진다. 대부분 전신 증상은 없지만, 큰 종기(carbuncle)의 경우에는 발열이나 오한, 몸살과 같은 전신 증상이 동반될 수도 있다. 


종기 증상
병명: 종기
통증이 가라앉는 경우도 있습니다 (야간 통곡).질병이 어느 정도 진행되면, 엄지 쪽에 감각이 떨어져 엄지 근육의 위약(쇠약) 및 위축이 나타나기도 하며, 손의 힘이 약해지고 손목을 잘 못쓰는 것과 같은 운동 마비 증세가 발생하기도 합니다. 손가락 및 손바닥이 부은 것 같은 부종감을 호소하나 일반적으로 실제적으로 부어있지는 않습니다. 찬물에 손을 넣거나 날씨가 추우면 손끝이 유난히 시리고 저린 증상도 흔히 관찰됩니다.
증상: 특징적인 증상은 손목 통증과 함께, 정중신경의 지배부위인 엄지, 검지 및 중지 및 손바닥 부위의 저림 증상이 밤에 심해지는 것입니다. 이러한 유발 증상이 있으면 수근관 증후군일 가능성이 더욱 높아집니다. 간혹 정중신경의 압박이 심한 경우 저림 및 감각 저하를 넘어 엄지 근육의 쇠약 및 위축이 나타나기도 합니다.증상은 통증과 감각이상 및 운동장애로 나누어 볼 수 있습니다. 엄지, 검지, 중지, 손바닥 부위에서 저리고 타는 듯한 통증과 손 저림, 이상 감각을 호소합니다. 통증 및 이상 감각의 성격은 불확실하고 모호한 경우도 있습니다. 1~2분 동안 손목을 굽히고 있을 때 해당 부